# More on Types 

In [1]:
struct Color{T} <: Number
    R::T
    G::T
    B::T
end

In [2]:
c1=Color{Int}(1,2,3)

Color{Int64}(1, 2, 3)

In [3]:
c2=Color{String}("1","2","3")

Color{String}("1", "2", "3")

In [4]:
typeof(c1)

Color{Int64}

In [5]:
typeof(c2)

Color{String}

In [6]:
typeof(c1)==typeof(c2)

false

In [7]:
subtypes(Number)

3-element Array{Any,1}:
 Color
 Complex
 Real

In [8]:
Color{Int} <: Integer

false

In [9]:
Color{Int} <: Number

true

# Binary file reading

In [1]:
open("../data/TEST.DAT") do io
    read(io,Float32) |> print
end

-2.3884733

### A syntactical interlude 

In [2]:
function myfunction(f,x,y,z)
    f(x)+f(y)+f(z)
end

myfunction (generic function with 1 method)

In [3]:
myfunction(x->x^2,1,2,2)

9

In [4]:
myfunction(1,2,2) do x
    (x+5)
end

20

### back to the main topic

In [5]:
open("../data/TEST.DAT") do io
    read(io,Float32) |> println
    read(io,Float32) |> println
end

-2.3884733
49.441597


In [6]:
"49.441597" |> sizeof

9

This files containts and number of records 
* long, lat (32bit float)
* speed (16bit Int) 
* COG (16 UInt)
* time (32 Int)
* data (elements 32 bit int) 6x600 (ch1, ch2....ch6)x600

In [7]:
r_size=(32*2+16+16+32+6*600*32)÷8

14416

In [8]:
n_rec=filesize("../data/TEST.DAT")÷r_size

636

In [9]:
struct RecHeader
    long::Float32
    lat::Float32
    speed::Int16
    COG::UInt16
    time::Int32
end

In [10]:
r=RecHeader(43,53,12,21,89)

RecHeader(43.0f0, 53.0f0, 12, 0x0015, 89)

In [11]:
isbits(r)

true

In [12]:
struct RecHeaderExtended 
    long::Float32
    lat::Float32
    speed::Int16
    COG::UInt16
    time::Int32
    data::Matrix{Int32}
end

In [14]:
r=RecHeaderExtended(43,53,12,21,89,[1 2; 3 4])

RecHeaderExtended(43.0f0, 53.0f0, 12, 0x0015, 89, Int32[1 2; 3 4])

In [15]:
isbits(r)

false

In [18]:
r.data[1]=15

15

In [20]:
r.data

2×2 Array{Int32,2}:
 15  2
  3  4

In [27]:
vcat(r.data,[60  90])

3×2 Array{Int64,2}:
 15   2
  3   4
 60  90

In [10]:
r=RecHeaderExtended(43,53,12,21,89,rand(Int32,600,6))

RecHeaderExtended(43.0f0, 53.0f0, 12, 0x0015, 89, Int32[1586954583 112079449 … -968118330 2065578325; 1137376555 483799436 … -302978788 1874647627; … ; 22341359 2081406909 … 404832656 1222586844; -2026256462 -95514093 … -1192113908 -601983024])

In [13]:
a=Array{RecHeader}(undef,2)
open("../data/TEST.DAT") do io
    read!(io, a) |> println
end

RecHeader[RecHeader(-2.3884733f0, 49.441597f0, 522, 0x0ced, 23269524), RecHeader(5.93531f-40, 5.97339f-40, -20017, 0x0006, 604280)]


In [24]:
open("../data/TEST.DAT") do io
    read!(io, Array{RecHeader}(undef,1)) |> println
end

RecHeader[RecHeader(-2.3884733f0, 49.441597f0, 522, 0x0ced, 23269524)]


In [25]:
a[2]

RecHeader(5.93531f-40, 5.97339f-40, -20017, 0x0006, 604280)

In [15]:
a=Array{RecHeaderExtended}(undef,2)

2-element Array{RecHeaderExtended,1}:
 #undef
 #undef

In [16]:
open("../data/TEST.DAT") do io
    read!(io, a) |> println
end

ErrorException: The IO stream does not support reading objects of type RecHeaderExtended.

In [17]:
function Base.read(io::IOStream, ::Type{RecHeaderExtended})
    long=read(io,Float32)
    lat=read(io,Float32)
    speed=read(io,Int16)
    COG=read(io,UInt16)
    time=read(io,Int32)
    data=Array{Int32}(undef,6,600)
    read!(io,data)
    RecHeaderExtended(long,lat,speed,COG,time, data)
end

In [18]:
open("../data/TEST.DAT") do io
    read(io, RecHeaderExtended) 
end

RecHeaderExtended(-2.3884733f0, 49.441597f0, 522, 0x0ced, 23269524, Int32[423558 423535 … 430269 430865; 426275 430871 … 432497 430643; … ; 442844 441355 … 442458 442234; 428231 428297 … 430647 431706])

In [19]:
my_data=Vector{RecHeaderExtended}(undef,n_rec)
read!("../data/TEST.DAT",my_data);

In [21]:
my_data

636-element Array{RecHeaderExtended,1}:
 RecHeaderExtended(-2.3884733f0, 49.441597f0, 522, 0x0ced, 23269524, Int32[423558 423535 … 430269 430865; 426275 430871 … 432497 430643; … ; 442844 441355 … 442458 442234; 428231 428297 … 430647 431706])
 RecHeaderExtended(-2.3886564f0, 49.441807f0, 519, 0x0ceb, 23270124, Int32[425857 424506 … 421696 427520; 429550 432356 … 438129 439917; … ; 442118 441354 … 440557 440668; 432302 432148 … 438045 438113])
 RecHeaderExtended(-2.38884f0, 49.442017f0, 520, 0x0ce9, 23270724, Int32[428584 425807 … 422405 425802; 438405 439045 … 438333 440640; … ; 439960 440564 … 439868 440399; 438390 437791 … 437512 437293])
 RecHeaderExtended(-2.3890233f0, 49.442226f0, 518, 0x0ceb, 23271324, Int32[425685 427883 … 426510 426448; 439707 437684 … 440979 437664; … ; 441711 442167 … 441356 441085; 437219 437114 … 439674 439868])
 RecHeaderExtended(-2.3892016f0, 49.44244f0, 515, 0x0cf4, 23271924, Int32[423784 426683 … 422322 427858; 442144 440565 … 441224 441583; … ; 440816

In [32]:
struct ChAcc{N} <: AbstractArray{Int32,1}
    d::Vector{RecHeaderExtended}
end

https://docs.julialang.org/en/v1/manual/interfaces/#man-interface-array-1

### Exercise 1

Create a wrapper around `my_data` to be able to acess a particular channel as and array

In [37]:
Base.size(ch::ChAcc{N}) where N = (length(ch.d)*(size(ch.d[1].data,2)),)

In [59]:
function Base.getindex(ch::ChAcc{N}, i) where N 
    n_read=size(ch.d[1].data,2)
    c=CartesianIndices((1:n_read,1:length(ch.d)))
    rec_num=c[i][2]
    ele_num=c[i][1]
    ch.d[rec_num].data[N,ele_num]
end

In [76]:
struct ChAccSimple <: AbstractArray{Int32,1}
    d::Vector{RecHeaderExtended}
    n::Int
    f::Int
    ChAccSimple(d::Vector{RecHeaderExtended},n::Int)=new(d,n,size(d[1].data,2))
end
Base.size(ch::ChAccSimple) = (length(ch.d)*ch.f,)
function Base.getindex(ch::ChAccSimple, i) 
    n_read=ch.f
    c=CartesianIndices((1:n_read,1:length(ch.d)))
    rec_num=c[i][2]
    ele_num=c[i][1]
    ch.d[rec_num].data[ch.n,ele_num]
end

In [79]:
channel_1=ChAccSimple(my_data,1);
all(ch1.==channel_1)

true

In [73]:
c=CartesianIndices((1:600,1:636))
i=700
(c[i][2],c[i][1])
a,b=c[i]

ErrorException: iteration is deliberately unsupported for CartesianIndex. Use `I` rather than `I...`, or use `Tuple(I)...`

In [70]:
ch1=ChAcc{1}(my_data);
ch2=ChAcc{2}(my_data);
typeof(ch1) ,typeof(ch2)

(ChAcc{1}, ChAcc{2})

In [ ]:
channel_1=ChAcc{1}(my_data)
plot(channel_1)
length(channel_1)==600*632 # should be true

### Exercise 2

Modify the function `Base.read(io::IOStream, ::Type{RecHeaderExtended})` to be more concices.. 
    should make use of field info.

In [78]:
struct BoatRecordM{F,NCH}  
    long::Float32
    lat::Float32
    speed::Int16
    COG::UInt16
    time::Int32
    data::Matrix{Int32}
    function BoatRecordM{F,NCH}(long,lat,speed,COG,time) where {F,NCH}
        new(long,lat,speed,COG,time,Matrix{Int32}(undef,F,NCH))
    end
end

# Do a consice read function 
# use fieldnames, fieldtype.... in one line you can read:long,lat,speed,COG,time...then construct you object... 
# then use `data` to read the rest of the record. 

In [80]:
r=BoatRecordM{500,4}(1,2,3,4,5)

BoatRecordM{500,4}(1.0f0, 2.0f0, 3, 0x0004, 5, Int32[65553 0 0 -8; 0 0 0 -1; … ; 0 81234768 -8 0; 0 0 -1 0])

In [81]:
r.data

500×4 Array{Int32,2}:
    65553         0           0          -8
        0         0           0          -1
        0  33554432           0           0
        0        65           0           0
   393234  47207152    47209488    47211472
        0         0           0           0
        0         0           1        4049
        0         0      327699           0
 47205488         0          31    62558448
        0         0           0           0
        0  47206824           0  -596246032
       16         0           0       32719
 47205376  47207752    47209520    62558448
        ⋮                        
 47207152        -8           0    50331648
        0        -1           0          65
        0         2           0    47213000
        0         0           0           0
        0      4112  -599013088    47213296
        0         0       32719           0
 33554432      6097           2    47213296
       57         0           0           0
 47207152  46633264 

In [ ]:
struct BoatRecordV{F,NCH}  
    long::Float32
    lat::Float32
    speed::Int16
    COG::UInt16
    time::Int32
    data::Vector{Vector{Int32}}
    function BoatRecordV{F,NCH}(long,lat,speed,COG,time) where {F,NCH}
        new(long,lat,speed,COG,time,
        [Vector{Int32}(undef,F) for i=1:NCH])
    end
end

In [ ]:
### First: Benchmark the reading of both approaches 
### Second: Benchmark the channel accessors for each 
ch1M=ChAccBM{1}(my_data)  ## ChAccBM to be implemented 
ch1V=ChAccBV{1}(my_data) ## ChAccBV to be implemented 
@benchmark exp.(ch1M)
@benchmark exp.(ch1V)


### Exercise 3

Test the performance of column based vs. row based representation of channels 

In [32]:
using BinaryRecReader


struct TestS
    a::Int16
    b::Float64
    m::Vector{Int32}
    k::UInt8 
end

vec_size=(600,1)

@construct_reader TestS vec_size

myrecs=Vector{TestS}(undef,15)
read!("somefile.txt",myrecs)

ArgumentError: ArgumentError: Package BinaryRecReader not found in current path:
- Run `import Pkg; Pkg.add("BinaryRecReader")` to install the BinaryRecReader package.
